In [1]:
print ("My Clustering Neighborhoods")

My Clustering Neighborhoods


In [ ]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

In [ ]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

toronto_df=pd.read_html(url, header=0)
toronto_df[0].to_csv('toronto.csv',index=False)
toronto_data=pd.read_csv('toronto.csv')

print (toronto_data.shape)
toronto_data.head()

In [ ]:
NABorough=toronto_data[toronto_data['Borough'] == 'Not assigned'].index
toronto_data.drop(NABorough, axis=0, inplace=True)
print(toronto_data.shape)
toronto_data.head()

In [ ]:
NANeigh=toronto_data[toronto_data['Neighbourhood']=='Not assigned'].index
NANeigh.shape

In [ ]:
print("Answer to Part 1 is",toronto_data.shape)

In [ ]:
!pip install pgeocode
import pgeocode
pgeocode.Nominatim('ca')
geolocator = pgeocode.Nominatim('ca')
postal_codes = toronto_data['Postal Code'].tolist()
latitudes = []
longitudes = []
print("pgeocode downloaded")

In [ ]:
for i, postal_code in enumerate(postal_codes):
    
    g = geolocator.query_postal_code(postal_code)
    
    if not g.empty:
        
        latitudes.append(g.latitude)
        longitudes.append(g.longitude)

In [ ]:
print (len(postal_codes),len(latitudes),len(longitudes))

toronto_data['Neighbourhood Latitude']=latitudes
toronto_data['Neighbourhood Longitude']=longitudes
toronto_data.head(103)

In [ ]:
ny = toronto_data[toronto_data['Borough'] == 'North York'].reset_index(drop=True)
print (ny.shape)
ny.head()

In [ ]:
CLIENT_ID = 'R12BNMHALD4Y5F1E5IWIA2YMYTTUZP4E0NZRUHVSMLA2WEIM' 
CLIENT_SECRET = 'UST5LXSGJNS0YH2ACP3JNUEIIUUIGOUONRC5VWZLZI5ZB3W0' 
VERSION = '20180605'
LIMIT = 100 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
ny_venues = getNearbyVenues(names = ny['Neighbourhood'], latitudes=ny['Neighbourhood Latitude'], longitudes=ny['Neighbourhood Longitude'])
print(ny_venues.shape) 
ny_venues.head()

In [ ]:
ny_venues.groupby('Neighbourhood').count() 
print('There are {} unique categories.'.format(len(ny_venues['Venue Category'].unique())))
                                                              
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

ny_onehot['Neighbourhood'] = ny_venues['Neighbourhood'] 

fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]
print(ny_onehot.shape)
ny_onehot.head()

In [ ]:
ny_grouped = ny_onehot.groupby('Neighbourhood').mean().reset_index()
print (ny_grouped.shape)  


num_top_venues = 5

for hood in ny_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

ny_venues_sorted = pd.DataFrame(columns=columns)
ny_venues_sorted['Neighbourhood'] = ny_grouped['Neighbourhood']

for ind in np.arange(ny_grouped.shape[0]):
    ny_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)
print(ny_venues.shape)
ny_venues_sorted.head()

In [ ]:
kclusters = 8

ny_grouped_clustering = ny_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)


kmeans.labels_[0:]

In [ ]:
ny_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = ny

ny_merged = ny_merged.join(ny_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

ny_merged.head()

In [ ]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

In [ ]:
map_clusters = folium.Map(location=[latitude,longitude],zoom_start=10)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Neighbourhood Latitude'], ny_merged['Neighbourhood Longitude'], ny_merged['Neighbourhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

In [ ]:
print("Cluster 2 is the most common cluster and has many Coffee Shops followed by Discount Stores.\n")